In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("coping_with_tsp.ipynb")

In [ ]:
# version shenanigans
!pip install -r requirements.txt --quiet
import otter
grader = otter.Notebook("coping_with_tsp.ipynb")
assert otter.__version__ >= "4.2.0", "Please restart your kernel."

In [ ]:
import pickle as pkl

# The Traveling Salesman Problem

In this notebook, we will take a crack at the Traveling Salesman Problem, which asks the following question: _Given a list of cities and the distances between each pair of cities, what is the shortest possible route that visits each city exactly once and returns to the origin city?_

The problem can be formally defined as follows:

```
Input: An n x n matrix of distances, where M[i, j] corresponds to the distance from city i to city j.
Output: An ordered list of cities [c1, c2, ... cn] that defines the shortest tour which passes through all the cities, 
starting and ending at the same city.
```

Though the question is conceptually simple, it is one of the **hardest problems in computer science!** In fact, it is an NP-hard problem. As such, we will not ask you to solve the traveling salesman problem in polynomial time. (If you do this, you have proven that P=NP!) Instead, we will develop efficient methods of approximating a solution, which is a common approach to tackling difficult problems.

We have provided a convenience function that, given an input matrix and a list of cities, evaluates the length of the path that passes through all of the cities in the list in order.

In [ ]:
def evaluate_path_length(matrix, path):
    """
    Evaluates the distance of the path using the weight matrix.
    
    Args:
        matrix: List[List[float]]
            An n x n matrix of distances, where M[i, j] corresponds to the distance from city i to city j.
        path: List[int] 
            A list corresponding to the order in which to visit cities, starting from path[0] and ending at path[-1] before 
            returning to path[0].
    
    Returns:
        path_length: int
            The length of the path.
    """
    
    path_length = 0
    for i in range(len(path)):
        path_length += matrix[path[i]][path[(i+1) % len(path)]]
    return path_length

### Q1a) Greedy Solution from Designated Home
Implement a greedy solution, which starts at city `home` and greedily chooses the closest city that has not been visited yet, until all cities have been visited. Return the path as a list of cities on that path, starting and ending at `path[0]`. For example, to represent the cycle `0 -> 1 -> 2 -> 3 -> 0`, return the list `[0, 1, 2, 3]`.  You may break ties arbitrarily.

In [ ]:
def tsp_greedy(matrix, home):
    """
    A greedy implementation of TSP, starting and ending at home.
    
    Args:
        matrix: List[List[float]]
            An n x n matrix of distances, where M[i, j] corresponds to the distance from city i to city j.
        home: int
            The index of the city to start and end at.
    
    Returns:
        path: List[int] 
            A list corresponding to the order in which to visit cities, starting from path[0] and ending at path[-1] before 
            returning to path[0]. path[0] should be home.    
    """
    ...

In [ ]:
grader.check("q1a")

### Q1b) Greedy Solution 
An easy way to improve over the original greedy solution is to try your greedy solution on all of the possible starting locations and choose the best one. Implement a general greedy solution, which runs the greedy implementation on all possible home locations, and returns the best overall path.

_Your solution should take around 8 lines of code. If you need an arbitrarily large value, you can use `1e9`_.

In [ ]:
def tsp_greedy_general(matrix):
    """
    A generalized greedy implementation of TSP.
    
    Args:
        matrix: List[List[float]]
            An n x n matrix of distances, where M[i, j] corresponds to the distance from city i to city j.
    
    Returns:
        path: List[int] 
            A list corresponding to the order in which to visit cities, starting from path[0] and ending at path[-1] before 
            returning to path[0].  
    """
    ...

In [ ]:
grader.check("q1b")

### Q2) Approximation Algorithm for Metric TSP

When difficult problems are given specific constraints, they are sometimes easier to approximate. For this question, we will focus on a special variant of TSP called the **metric TSP**, which satisfies the following inequality:

$$ \forall i, j, k \in V,\; d(i, k) \leq d(i, j) + d(j, k)$$

_(This is called the triangle inequality, and all mathematical distance metrics obey it, which is why this is called the metric TSP!)_

In other words, the shortest path from one city to another city is always the direct path.

The Metric TSP problem is still NP-hard, but the following approximation returns a path that is guaranteed to be **at most twice the length of the optimal path**:

* Generate a minimum spanning tree of the graph.
* Run depth-first search on the minimum spanning tree.
* Return the nodes in the order that you found them with depth first search (i.e. by preorder number).

We have provided an implementation of Kruskal's algorithm and other utilities you may find useful.

In [ ]:
class UnionFind:
    """Implements the UnionFind data structure with n nodes. Useful for running Kruskal's algorithm."""
    def __init__(self, n):
        self.n = n
        self.parents = [i for i in range(n)]
        self.rank = [1]*n

    def find(self, i):
        """Returns the root ancestor of node i."""
        assert i >= 0 and i <= self.n-1, f"Node {i} is not in the data structure. Only nodes {0} through {self.n-1} exist."
        if i != self.parents[i]:
            self.parents[i] = self.find(self.parents[i])
        return self.parents[i]

    def union(self, i, j):
        """Merges the trees containing nodes i and j."""
        assert i >= 0 and i <= self.n-1, f"Node {i} is not in the data structure. Only nodes {0} through {self.n-1} exist."
        assert j >= 0 and j <= self.n-1, f"Node {j} is not in the data structure. Only nodes {0} through {self.n-1} exist."
        pi, pj = self.find(i), self.find(j)
        if pi != pj:
            if self.rank[pi] < self.rank[pj]:
                self.parents[pi] = pj
            elif self.rank[pi] > self.rank[pj]:
                self.parents[pi] = pi
            else:
                self.parents[pi] = pj
                self.rank[pi] += 1

def kruskal(matrix):
    """
    Runs Kruskal's MST-finding algorithm on an adjacency-matrix representation of a graph.
    Args:
        matrix: List[List[int]]
            An adjacency matrix representation of a weighted graph, where matrix[i][j] corresponds to the weight of edge (i, j).
    Returns:
        T: List[Tuple[int]]
            An edge list representation of the resulting MST, in which each element is a pair of nodes connected by an edge.
    """
    T = []
    edges = []
    for u in range(len(matrix)):
        for v, w in enumerate(matrix[u]):
            edges.append((w, u, v))
    edges.sort()
    UF = UnionFind(len(matrix))
    for e in edges:
        u, v = e[1], e[2]
        if UF.find(u) != UF.find(v):
            UF.union(u, v)
            T.append((u, v))
    if len(T) != len(matrix) - 1:
        return None
    return T

def make_adj_list(n, edge_list):
    """
    Converts an edge list into an adjacency list.
    Args:
        n: int
            The number of nodes in the graph.
        edge_list: List[List[int]]
            An edge list representation of a graph, where each element is a pair of nodes connected by an edge. 
            For instance, (2)--(0)--(1) would be represented as [[0, 1], [0, 2]].
    Returns:
        adj_list: List[List[int]]
            An adjacency list representation of a graph, where adj_list[i] lists all of the nodes with an edge to node i. 
            For instance, (2)--(0)--(1) would be represented as [[1, 2], [0], [0]].
    """
    adj_list = [[] for i in range(0,n)] 
    for edge in edge_list:
        adj_list[edge[0]].append(edge[1]) # need to include both directions for the edge
        adj_list[edge[1]].append(edge[0])
    return adj_list

First, implement depth-first-search. You may assume that the input graph is connected.

In [ ]:
def preorder(adj_list, s):
    """
    Runs depth-first-search on the graph, and returns the list of nodes sorted by preorder.
    
    Args:
        adj_list: List[List[int]]
            An adjacency list representation of a graph, where adj_list[i] lists all of the nodes with an edge to node i. 
            For instance, (2)--(0)--(1) would be represented as [[1, 2], [0], [0]].
        s: int
            The node from which to start DFS.
    
    Returns:
        preorder: List[int]
            The list of nodes sorted by preorder.
            
    """
    ...

In [ ]:
grader.check("q2a")

Now, implement the metric TSP approximation algorithm. **Run DFS starting at node 0 to ensure compatibility with the autograder.**

In [ ]:
def metric_tsp_approximation(matrix):
    """
    An algorithm for solving the Metric TSP using minimum spanning trees and depth first search.
    
    Args:
        matrix: List[List[float]]
            An n x n matrix of distances, where M[i, j] corresponds to the distance from city i to city j.
    
    Returns:
        path: List[int] 
            A list corresponding to the order in which to visit cities, starting from path[0] and ending at path[-1] before 
            returning to path[0].  
    """
    ...

To help you debug, we have provided some sample test cases and answers. *Passing all these tests does not ensure the infallibility of your solution.*

In [ ]:
with open('tests_1.pkl', 'rb') as f:
    cases = pkl.load(f)

def standardize_path(path):
    idx = path.index(0)
    return path[idx:] + path[:idx]

def matrix_to_string(matrix):
    return "".join([f"{row}\n" for row in matrix])
                   
def verify_basic(matrix, path):
    """Verify that the proposed solution is valid."""
    assert len(path) == len(matrix), f"There are {len(matrix)} cities but your path only has {len(path)} cities!"
    assert sorted(path) == list(range(len(path))), f"Your path is not a permutation of cities (ints from 0 to {len(path)})"

for problem, solution in cases:
    your_solution = metric_tsp_approximation(m)
    verify_basic(problem, your_solution)
    assert standardize_path(your_solution) == standardize_path(solution), (
        f"Your solution returned the incorrect path!"

        f"The matrix: \n{matrix_to_string(m)}\n"
        f"The path you returned: \n{student_sol}"
        f"The correct path: \n{s}")
print("All test cases passed!")

In [ ]:
grader.check("q2b")

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit.

In [ ]:
grader.export(pdf=False, force_save=True, run_tests=True)